In [2]:
def one_n_rest(iterable):

    # one_n_rest("ABC") -> (A, BC), (B, AC), (C, AB)
    for one_i, one in enumerate(iterable):
        yield one, (rest for rest_i, rest in enumerate(iterable) if rest_i != one_i)



test = one_n_rest("ABC")

for one, rest in test:
    for thing in rest:
        print (one, thing)
    print ("")

A B
A C

B A
B C

C A
C B



In [62]:
import re
from collections import defaultdict
from pprint import pp


sample = open('d19.sample').read()

scanner_find = re.compile(r'--- scanner (\d?) ---\s+(.*?)(?=\Z|\s{2,})', flags=re.S|re.M).findall

coord_find = re.compile(r'(-?\d+),(-?\d+),(-?\d+)').findall


def get_data(data):
    results = defaultdict(list)
    for scanner_ids, coords in scanner_find(data):
        scanner_id = int(scanner_ids)
        for x,y,z in coord_find(coords):
            results[scanner_id].append((int(x),int(y),int(z)))
    return dict(results)
            


results = get_data(sample)


distances = defaultdict(list)

for scanner, coords in results.items():
    for (x1,y1,z1), remaining in one_n_rest(coords):
        for x2,y2,z2 in remaining:
            distances[scanner].append(((x2 - x1) ** 2) + ((y2 - y1) ** 2) + ((z2 - z1) ** 2))


from collections import Counter
completed = []
for scanner, others in one_n_rest(distances):
    for other in others:
        id = tuple(sorted([scanner, other]))
        if id in completed:
            continue
        completed.append(id)
        scanner_distances = distances[scanner]
        other_distances = distances[other]
        counts = Counter(list(set(scanner_distances)) + list(set(other_distances)))
        commons = len([c for c in counts.values() if c > 1])
        if commons >= 66:
            print(scanner, other, commons)
        else:
            print(scanner, other)




            

0 1 66
0 2
0 3
0 4
1 2
1 3 66
1 4 66
2 3
2 4 66
3 4
